In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from pickle import dump
from sklearn.metrics import mean_squared_error, r2_score
import math

In [2]:
y_train = pd.read_csv('/workspaces/EDA_4/data/processed/heart_prevalence_y_train.csv')
y_test = pd.read_csv('/workspaces/EDA_4/data/processed/heart_prevalence_y_test.csv')
X_train_norm = pd.read_csv('/workspaces/EDA_4/data/interim/heart_prevalence_X_train_std.csv')
X_test_norm = pd.read_csv('/workspaces/EDA_4/data/interim/heart_prevalence_X_test_std.csv')

In [3]:
from sklearn.feature_selection import f_regression, SelectKBest

In [4]:
rmss = []
r2s = []
for p in [1, 0.8, 0.7, 0.6, 0.5]:

    selection_model = SelectKBest(f_regression, k = int(len(X_train_norm.columns) * p))
    selection_model.fit(X_train_norm, y_train)
    ix = selection_model.get_support()

    X_train_sel = pd.DataFrame(selection_model.transform(X_train_norm), columns = X_train_norm.columns.values[ix])
    X_test_sel = pd.DataFrame(selection_model.transform(X_test_norm), columns = X_test_norm.columns.values[ix])

    dump(selection_model, open(f"/workspaces/EDA_4/models/selection_model{p}.pk", "wb"))

    X_train_sel["Heart disease_prevalence"] = y_train
    X_test_sel["Heart disease_prevalence"] = y_test 
    train_data = X_train_sel
    test_data = X_test_sel

    X_train = train_data.drop(["Heart disease_prevalence"], axis = 1)
    y_train = train_data["Heart disease_prevalence"]
    X_test = test_data.drop(["Heart disease_prevalence"], axis = 1)
    y_test = test_data["Heart disease_prevalence"]  

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmss.append(math.sqrt(mean_squared_error(y_test, y_pred)))
    r2s.append(r2_score(y_test, y_pred))

best_rmss = rmss.index(max(rmss)) 
best_r2s = r2s.index(max(r2s)) 

print(f"Error cuadrático medio: {rmss[best_rmss]}")
print(f"Coeficiente de determinación: {r2s[best_r2s]}")


    

Error cuadrático medio: 0.5255850187397291
Coeficiente de determinación: 0.9599941816299377


/home/vscode/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
X_train_sel["Heart disease_prevalence"] = y_train
X_test_sel["Heart disease_prevalence"] = y_test 

In [6]:
X_train_sel.to_csv("/workspaces/EDA_4/data/interim/heart_prevalence_X_train_sel.csv", index=False)
X_test_sel.to_csv("/workspaces/EDA_4/data/interim/heart_prevalence_X_test_sel.csv", index=False)

In [7]:
train_data = X_train_sel
test_data = X_test_sel
train_data.head()

,6,8,9,10,11,12,Heart disease_prevalence
0,0.820420,-1.254632,-1.034310,-0.690028,-1.329247,-1.063396,7.321739
1,0.113865,-0.126974,-0.141938,-0.158183,-0.433800,-0.535918,8.031343
2,-0.311410,0.363973,0.936350,-0.307709,0.011809,-0.196489,8.315238
3,1.409531,-1.011609,-1.099331,-0.350395,-1.043889,-1.534241,7.230000
4,-1.292706,0.003443,-0.337835,-0.487564,0.924241,1.998352,8.663636


In [8]:
X_train = train_data.drop(["Heart disease_prevalence"], axis = 1)
y_train = train_data["Heart disease_prevalence"]
X_train.head()

,6,8,9,10,11,12
0,0.820420,-1.254632,-1.034310,-0.690028,-1.329247,-1.063396
1,0.113865,-0.126974,-0.141938,-0.158183,-0.433800,-0.535918
2,-0.311410,0.363973,0.936350,-0.307709,0.011809,-0.196489
3,1.409531,-1.011609,-1.099331,-0.350395,-1.043889,-1.534241
4,-1.292706,0.003443,-0.337835,-0.487564,0.924241,1.998352


In [9]:
X_test = test_data.drop(["Heart disease_prevalence"], axis = 1)
y_test = test_data["Heart disease_prevalence"]
X_test.head()

,6,8,9,10,11,12
0,2.333274,-0.312357,-0.103458,-0.745705,-0.220811,-1.029929
1,3.201860,-2.592387,-2.434579,-2.193322,-2.465568,-2.489778
2,-0.023037,-0.233735,-0.412420,0.063166,-0.085710,0.403377
3,0.546627,-1.980160,-2.283529,-1.204175,-1.573678,-0.723029
4,-0.386932,-0.434702,-0.682066,0.068579,-0.628342,-0.081028


In [10]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [11]:
print(f"Intercept (a): {model.intercept_}")
print(f"Coefficients (b): {model.coef_}")

Intercept (a): 8.371498647349844
Coefficients (b): [-0.15549461  0.75459943 -0.27330536  0.53737403 -0.61739438  0.56593799]


In [12]:
y_pred = model.predict(X_test)
y_pred

array([6.95398743, 5.51733046, 8.6265677 , 7.33167183, 8.66898132,
       9.07482422, 9.56759501, 6.13899544, 8.10306919, 6.4907785 ,
       8.07155428])

In [13]:


print(f"Error cuadrático medio: {math.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred)}")

Error cuadrático medio: 0.5255850187397291
Coeficiente de determinación: 0.8686706799451416
